In [1]:
import numpy as np
#import pyeeg as pe
import pickle as pickle
import pandas as pd
import math

from sklearn import svm
from sklearn.preprocessing import StandardScaler 
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

import os
import tensorflow as tf
import time
from sklearn.metrics import accuracy_score, f1_score
from sklearn.svm import SVC
from sklearn.metrics import classification_report


## 4.Regression
### 0. Loading Training and Testing dataset

In [2]:
with open('C:/Users/wongs/OneDrive/Documents/dissertation/DEAP/out_psds_data_training_2s.npy', 'rb') as fileTrain:
    X_2s  = np.load(fileTrain)
    
with open('C:/Users/wongs/OneDrive/Documents/dissertation/DEAP/out_psds_label_training_2s.npy', 'rb') as fileTrainL:
    Y_2s  = np.load(fileTrainL)

ss = StandardScaler()
X_2s = ss.fit_transform(X_2s)
Z_2s = np.ravel(Y_2s[:, [1]])

Arousal_Train_2s = np.ravel(Y_2s[:, [0]])
Valence_Train_2s = np.ravel(Y_2s[:, [1]])
Domain_Train_2s = np.ravel(Y_2s[:, [2]])
Like_Train_2s = np.ravel(Y_2s[:, [3]])
print(Arousal_Train_2s)


with open('C:/Users/wongs/OneDrive/Documents/dissertation/DEAP/out_psds_data_testing_2s.npy', 'rb') as fileTrain:
    M_2s  = np.load(fileTrain)
    
with open('C:/Users/wongs/OneDrive/Documents/dissertation/DEAP/out_psds_label_testing_2s.npy', 'rb') as fileTrainL:
    N_2s  = np.load(fileTrainL)

M_2s = ss.transform(M_2s)
L_2s = np.ravel(N_2s[:, [1]])

Arousal_Test_2s = np.ravel(N_2s[:, [0]])
Valence_Test_2s = np.ravel(N_2s[:, [1]])
Domain_Test_2s = np.ravel(N_2s[:, [2]])
Like_Test_2s = np.ravel(N_2s[:, [3]])

[5.12 3.05 3.12 ... 6.03 3.17 6.99]


In [3]:
# Binarize Arousal_Test_2s with threshold 5
Arousal_Test_2s_binary = np.where(Arousal_Test_2s >= 5, 1, 0)
Arousal_Train_2s_binary = np.where(Arousal_Train_2s >= 5, 1, 0)
# Binarize Valence_Train_2s with threshold 5
Valence_Train_2s_binary = np.where(Valence_Train_2s >= 5, 1, 0)
# Binarize Valence_Test_2s with threshold 5
Valence_Test_2s_binary = np.where(Valence_Test_2s >= 5, 1, 0)
# Binarize Domain_Test_2s with threshold 5
Domain_Test_2s_binary = np.where(Domain_Test_2s >= 5, 1, 0)
# Binarize Like_Train_2s with threshold 5
Domain_Train_2s_binary = np.where(Domain_Train_2s >= 5, 1, 0)

In [4]:
svmrbf_arousal_2s_psds = SVC(kernel='rbf',max_iter = 100000,random_state=42)
svmlinear_arousal_2s_psds = SVC(kernel='linear',max_iter=100000,random_state=42)

# Train the models on the GPU
svmrbf_arousal_2s_psds.fit(X_2s, Arousal_Train_2s_binary)
svmlinear_arousal_2s_psds.fit(X_2s, Arousal_Train_2s_binary)

# Predict on new data
predictions_rbf = svmrbf_arousal_2s_psds.predict(M_2s)
predictions_linear = svmlinear_arousal_2s_psds.predict(M_2s)

# Evaluate the models
svmrbf_arousal_2s_psds_accuracy = accuracy_score(Arousal_Test_2s_binary, predictions_rbf)
svmrbf_arousal_2s_psds_f1 = f1_score(Arousal_Test_2s_binary, predictions_rbf, average='weighted')

svmlinear_arousal_2s_psds_accuracy = accuracy_score(Arousal_Test_2s_binary, predictions_linear)
svmlinear_arousal_2s_psds_f1 = f1_score(Arousal_Test_2s_binary, predictions_linear, average='weighted')

print("RBF SVM:")
print(f"Accuracy: {svmrbf_arousal_2s_psds_accuracy * 100:.2f}%")
print(f"F1 Score: {svmrbf_arousal_2s_psds_f1:.2f}")

print("Linear SVM:")
print(f"Accuracy: {svmlinear_arousal_2s_psds_accuracy * 100:.2f}%")
print(f"F1 Score: {svmlinear_arousal_2s_psds_f1:.2f}")


C:\Users\wongs\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\svm\_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=100000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


RBF SVM:
Accuracy: 64.66%
F1 Score: 0.63
Linear SVM:
Accuracy: 55.54%
F1 Score: 0.48


In [5]:
svmrbf_valence_2s_psds = SVC(kernel='rbf',max_iter = 100000,random_state=42)
svmlinear_valence_2s_psds = SVC(kernel='linear',max_iter = 100000,random_state=42)

# Train the models on the GPU
svmrbf_valence_2s_psds.fit(X_2s, Valence_Train_2s_binary)
svmlinear_valence_2s_psds.fit(X_2s, Valence_Train_2s_binary)

# Predict on new data
predictions_rbf = svmrbf_valence_2s_psds.predict(M_2s)
predictions_linear = svmlinear_valence_2s_psds.predict(M_2s)

# Evaluate the models
svmrbf_valence_2s_psds_accuracy = accuracy_score(Valence_Test_2s_binary, predictions_rbf)
svmrbf_valence_2s_psds_f1 = f1_score(Valence_Test_2s_binary, predictions_rbf, average='weighted')

svmlinear_valence_2s_psds_accuracy = accuracy_score(Valence_Test_2s_binary, predictions_linear)
svmlinear_valence_2s_psds_f1 = f1_score(Valence_Test_2s_binary, predictions_linear, average='weighted')

print("RBF SVM:")
print(f"Accuracy: {svmrbf_valence_2s_psds_accuracy * 100:.2f}%")
print(f"F1 Score: {svmrbf_valence_2s_psds_f1:.2f}")

print("Linear SVM:")
print(f"Accuracy: {svmlinear_valence_2s_psds_accuracy * 100:.2f}%")
print(f"F1 Score: {svmlinear_valence_2s_psds_f1:.2f}")

C:\Users\wongs\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\svm\_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=100000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


RBF SVM:
Accuracy: 66.55%
F1 Score: 0.65
Linear SVM:
Accuracy: 58.41%
F1 Score: 0.59


In [6]:
# Initialize the SVM models with RBF and linear kernels
svmrbf_domain_2s_psds = SVC(kernel='rbf',max_iter = 100000,random_state=42)
svmlinear_domain_2s_psds = SVC(kernel='linear',max_iter = 100000,random_state=42)

# Train the models on the GPU
svmrbf_domain_2s_psds.fit(X_2s, Domain_Train_2s_binary)
svmlinear_domain_2s_psds.fit(X_2s, Domain_Train_2s_binary)

# Predict on new data
predictions_rbf = svmrbf_domain_2s_psds.predict(M_2s)
predictions_linear = svmlinear_domain_2s_psds.predict(M_2s)

# Evaluate the models
svmrbf_domain_2s_psds_accuracy = accuracy_score(Domain_Test_2s_binary, predictions_rbf)
svmrbf_domain_2s_psds_f1 = f1_score(Domain_Test_2s_binary, predictions_rbf, average='weighted')

svmlinear_domain_2s_psds_accuracy = accuracy_score(Domain_Test_2s_binary, predictions_linear)
svmlinear_domain_2s_psds_f1 = f1_score(Domain_Test_2s_binary, predictions_linear, average='weighted')

print("RBF SVM:")
print(f"Accuracy: {svmrbf_domain_2s_psds_accuracy * 100:.2f}%")
print(f"F1 Score: {svmrbf_domain_2s_psds_f1:.2f}")

print("Linear SVM:")
print(f"Accuracy: {svmlinear_domain_2s_psds_accuracy * 100:.2f}%")
print(f"F1 Score: {svmlinear_domain_2s_psds_f1:.2f}")

C:\Users\wongs\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\svm\_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=100000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


RBF SVM:
Accuracy: 67.78%
F1 Score: 0.66
Linear SVM:
Accuracy: 65.93%
F1 Score: 0.63


In [7]:
from enum import Enum
class Norm(Enum):
    Manhattan = 1
    Euclidean = 2
n     = 5
norm  = Norm.Euclidean.value
from sklearn import neighbors, linear_model
from sklearn.metrics import classification_report
knn = neighbors.KNeighborsClassifier(n_neighbors=n, weights='uniform', p=norm)

In [8]:
print(int(math.sqrt(len(X_2s))))


249


In [9]:
n = int(math.sqrt(len(X_2s)))
knn = neighbors.KNeighborsClassifier(n_neighbors=n, weights='uniform', p=norm)
knn_arousal_psds = knn.fit(X_2s, Arousal_Train_2s_binary)
print('k_NN accuracy for test set: %f' % knn_arousal_psds.score(M_2s, Arousal_Test_2s_binary))
y_true, y_pred = Arousal_Test_2s_binary, knn_arousal_psds.predict(M_2s)
print(classification_report(y_true, y_pred))

k_NN accuracy for test set: 0.649078
              precision    recall  f1-score   support

           0       0.62      0.50      0.55      6829
           1       0.66      0.77      0.71      8787

    accuracy                           0.65     15616
   macro avg       0.64      0.63      0.63     15616
weighted avg       0.65      0.65      0.64     15616



In [10]:

knn_valence_psds = knn.fit(X_2s, Valence_Train_2s_binary)
print('k_NN accuracy for test set: %f' % knn_valence_psds.score(M_2s, Valence_Test_2s_binary))
y_true, y_pred = Valence_Test_2s_binary, knn_valence_psds.predict(M_2s)
print(classification_report(y_true, y_pred))

k_NN accuracy for test set: 0.681096
              precision    recall  f1-score   support

           0       0.66      0.47      0.55      6433
           1       0.69      0.83      0.75      9183

    accuracy                           0.68     15616
   macro avg       0.67      0.65      0.65     15616
weighted avg       0.68      0.68      0.67     15616



In [11]:
knn_domain_psds = knn.fit(X_2s, Domain_Train_2s_binary)
print('k_NN accuracy for test set: %f' % knn_domain_psds.score(M_2s, Domain_Test_2s_binary))
y_true, y_pred = Domain_Test_2s_binary, knn_domain_psds.predict(M_2s)
print(classification_report(y_true, y_pred))

k_NN accuracy for test set: 0.684298
              precision    recall  f1-score   support

           0       0.63      0.43      0.51      5967
           1       0.70      0.84      0.77      9649

    accuracy                           0.68     15616
   macro avg       0.67      0.64      0.64     15616
weighted avg       0.67      0.68      0.67     15616



In [12]:
Val_R_2s = RandomForestRegressor(n_estimators=512, n_jobs=6,random_state=42)
Val_R_2s.fit(X_2s, Valence_Train_2s_binary)
Valence_pred_2s = Val_R_2s.predict(M_2s)


In [13]:
Valence_pred_2s = np.where(Valence_pred_2s >= 0.5, 1, 0)

In [14]:
Ran_valence_2s_psds_accuracy = accuracy_score(Valence_Test_2s_binary, Valence_pred_2s)
Ran_valence_2s_psds_f1 = f1_score(Valence_Test_2s_binary, Valence_pred_2s, average='weighted')

In [15]:
print(Ran_valence_2s_psds_accuracy,Ran_valence_2s_psds_f1)

0.7898309426229508 0.7863206849132771


In [16]:
Aro_R_2s = RandomForestRegressor(n_estimators=512, n_jobs=6,random_state=42)
Aro_R_2s.fit(X_2s, Arousal_Train_2s_binary)
Arousal_pred_2s = Aro_R_2s.predict(M_2s)

Arousal_pred_2s = np.where(Arousal_pred_2s >= 0.5, 1, 0)
Ran_Arousal_2s_psds_accuracy = accuracy_score(Arousal_Test_2s_binary, Arousal_pred_2s)
Ran_Arousal_2s_psds_f1 = f1_score(Arousal_Test_2s_binary, Arousal_pred_2s, average='weighted')

print(Ran_Arousal_2s_psds_accuracy,Ran_Arousal_2s_psds_f1)

0.7749103483606558 0.7730074830937491


In [17]:
Dom_R_2s = RandomForestRegressor(n_estimators=512, n_jobs=6,random_state=42)
Dom_R_2s.fit(X_2s, Domain_Train_2s_binary)
Domain_pred_2s = Dom_R_2s.predict(M_2s)

Domain_pred_2s = np.where(Domain_pred_2s >= 0.5, 1, 0)
Ran_Domain_2s_psds_accuracy = accuracy_score(Domain_Test_2s_binary, Domain_pred_2s)
Ran_Domain_2s_psds_f1 = f1_score(Domain_Test_2s_binary, Domain_pred_2s, average='weighted')

print(Ran_Domain_2s_psds_accuracy,Ran_Domain_2s_psds_f1)

0.7887423155737705 0.7849876518889632


In [18]:
with open('C:/Users/wongs/OneDrive/Documents/dissertation/DEAP/out_ica_data_training_2s.npy', 'rb') as fileTrain:
    X_2s_ica  = np.load(fileTrain)
    
with open('C:/Users/wongs/OneDrive/Documents/dissertation/DEAP/out_ica_label_training_2s.npy', 'rb') as fileTrainL:
    Y_2s_ica  = np.load(fileTrainL)

ss = StandardScaler()
X_2s_ica = ss.fit_transform(X_2s_ica)
Z_2s_ica = np.ravel(Y_2s_ica[:, [1]])

Arousal_Train_ica = np.ravel(Y_2s_ica[:, [0]])
Valence_Train_ica = np.ravel(Y_2s_ica[:, [1]])
Domain_Train_ica = np.ravel(Y_2s_ica[:, [2]])
Like_Train_ica = np.ravel(Y_2s_ica[:, [3]])



with open('C:/Users/wongs/OneDrive/Documents/dissertation/DEAP/out_ica_data_testing_2s.npy', 'rb') as fileTrain:
    M_2s_ica  = np.load(fileTrain)
    
with open('C:/Users/wongs/OneDrive/Documents/dissertation/DEAP/out_ica_label_testing_2s.npy', 'rb') as fileTrainL:
    N_2s_ica  = np.load(fileTrainL)

M_2s_ica = ss.transform(M_2s_ica)
L_2s_ica = np.ravel(N_2s_ica[:, [1]])

Arousal_Test_ica = np.ravel(N_2s_ica[:, [0]])
Valence_Test_ica = np.ravel(N_2s_ica[:, [1]])
Domain_Test_ica = np.ravel(N_2s_ica[:, [2]])
Like_Test_ica = np.ravel(N_2s_ica[:, [3]])

# Binarize Arousal_Test_2s with threshold 5
Arousal_Test_ica_binary = np.where(Arousal_Test_ica >= 5, 1, 0)
Arousal_Train_ica_binary = np.where(Arousal_Train_ica >= 5, 1, 0)
# Binarize Valence_Train_2s with threshold 5
Valence_Train_ica_binary = np.where(Valence_Train_ica >= 5, 1, 0)
# Binarize Valence_Test_2s with threshold 5
Valence_Test_ica_binary = np.where(Valence_Test_ica >= 5, 1, 0)
# Binarize Domain_Test_2s with threshold 5
Domain_Test_ica_binary = np.where(Domain_Test_ica >= 5, 1, 0)
# Binarize Like_Train_2s with threshold 5
Domain_Train_ica_binary = np.where(Domain_Train_ica >= 5, 1, 0)

In [19]:
svmlinear_arousal_2s_ica = SVC(kernel='linear',max_iter=100000,random_state=42,verbose=True)
svmlinear_arousal_2s_ica.fit(X_2s_ica, Arousal_Train_ica_binary)
predictions_linear_arousal = svmlinear_arousal_2s_ica.predict(M_2s_ica)


svmlinear_arousal_2s_ica_accuracy = accuracy_score(Arousal_Test_ica_binary, predictions_linear_arousal)
svmlinear_arousal_2s_ica_f1 = f1_score(Arousal_Test_ica_binary, predictions_linear_arousal, average='weighted')
print("Arousal Linear SVM:")
print(f"Accuracy: {svmlinear_arousal_2s_ica_accuracy * 100:.2f}%")
print(f"F1 Score: {svmlinear_arousal_2s_ica_f1:.2f}")


[LibSVM]

C:\Users\wongs\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\svm\_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=100000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


Arousal Linear SVM:
Accuracy: 55.91%
F1 Score: 0.43


In [20]:
svmrbf_arousal_2s_ica = SVC(kernel='rbf',max_iter=100000,random_state=42,verbose=True)

# Train the models
svmrbf_arousal_2s_ica.fit(X_2s_ica, Arousal_Train_ica_binary)

# Predict on new data
predictions_rbf_arousal = svmrbf_arousal_2s_ica.predict(M_2s_ica)


# Evaluate the models
svmrbf_arousal_2s_ica_accuracy = accuracy_score(Arousal_Test_ica_binary, predictions_rbf_arousal)
svmrbf_arousal_2s_ica_f1 = f1_score(Arousal_Test_ica_binary, predictions_rbf_arousal, average='weighted')



# Print the results
print("Arousal RBF SVM:")
print(f"Accuracy: {svmrbf_arousal_2s_ica_accuracy * 100:.2f}%")
print(f"F1 Score: {svmrbf_arousal_2s_ica_f1:.2f}")




[LibSVM]

C:\Users\wongs\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\svm\_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=100000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


Arousal RBF SVM:
Accuracy: 92.76%
F1 Score: 0.93


In [21]:
svmrbf_valence_2s_ica = SVC(kernel='rbf',max_iter=100000,random_state=42,verbose=True)
svmlinear_valence_2s_ica = SVC(kernel='linear',max_iter=100000,random_state=42,verbose=True)

svmrbf_valence_2s_ica.fit(X_2s_ica, Valence_Train_ica_binary)
svmlinear_valence_2s_ica.fit(X_2s_ica, Valence_Train_ica_binary)

predictions_rbf_valence = svmrbf_valence_2s_ica.predict(M_2s_ica)
predictions_linear_valence = svmlinear_valence_2s_ica.predict(M_2s_ica)

svmrbf_valence_2s_ica_accuracy = accuracy_score(Valence_Test_ica_binary, predictions_rbf_valence)
svmrbf_valence_2s_ica_f1 = f1_score(Valence_Test_ica_binary, predictions_rbf_valence, average='weighted')

svmlinear_valence_2s_ica_accuracy = accuracy_score(Valence_Test_ica_binary, predictions_linear_valence)
svmlinear_valence_2s_ica_f1 = f1_score(Valence_Test_ica_binary, predictions_linear_valence, average='weighted')

print("Valence RBF SVM:")
print(f"Accuracy: {svmrbf_valence_2s_ica_accuracy * 100:.2f}%")
print(f"F1 Score: {svmrbf_valence_2s_ica_f1:.2f}")

print("Valence Linear SVM:")
print(f"Accuracy: {svmlinear_valence_2s_ica_accuracy * 100:.2f}%")
print(f"F1 Score: {svmlinear_valence_2s_ica_f1:.2f}")


[LibSVM]

C:\Users\wongs\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\svm\_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=100000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


[LibSVM]

C:\Users\wongs\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\svm\_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=100000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


Valence RBF SVM:
Accuracy: 91.01%
F1 Score: 0.91
Valence Linear SVM:
Accuracy: 58.78%
F1 Score: 0.46


In [22]:
svmrbf_domain_2s_ica = SVC(kernel='rbf',max_iter=100000,random_state=42,verbose=True)
svmlinear_domain_2s_ica = SVC(kernel='linear',max_iter=100000,random_state=42,verbose=True)

svmrbf_domain_2s_ica.fit(X_2s_ica, Domain_Train_ica_binary)

svmlinear_domain_2s_ica.fit(X_2s_ica, Domain_Train_ica_binary)

predictions_rbf_domain = svmrbf_domain_2s_ica.predict(M_2s_ica)
predictions_linear_domain = svmlinear_domain_2s_ica.predict(M_2s_ica)

svmrbf_domain_2s_ica_accuracy = accuracy_score(Domain_Test_ica_binary, predictions_rbf_domain)
svmrbf_domain_2s_ica_f1 = f1_score(Domain_Test_ica_binary, predictions_rbf_domain, average='weighted')

svmlinear_domain_2s_ica_accuracy = accuracy_score(Domain_Test_ica_binary, predictions_linear_domain)
svmlinear_domain_2s_ica_f1 = f1_score(Domain_Test_ica_binary, predictions_linear_domain, average='weighted')

print("Domain RBF SVM:")
print(f"Accuracy: {svmrbf_domain_2s_ica_accuracy * 100:.2f}%")
print(f"F1 Score: {svmrbf_domain_2s_ica_f1:.2f}")

print("Domain Linear SVM:")
print(f"Accuracy: {svmlinear_domain_2s_ica_accuracy * 100:.2f}%")
print(f"F1 Score: {svmlinear_domain_2s_ica_f1:.2f}")

[LibSVM]

C:\Users\wongs\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\svm\_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=100000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


[LibSVM]

C:\Users\wongs\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\svm\_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=100000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


Domain RBF SVM:
Accuracy: 92.86%
F1 Score: 0.93
Domain Linear SVM:
Accuracy: 60.77%
F1 Score: 0.51


In [23]:
n = int(math.sqrt(len(X_2s_ica)))
knn = neighbors.KNeighborsClassifier(n_neighbors=n, weights='uniform', p=norm)
knn_arousal_ica = knn.fit(X_2s_ica, Arousal_Train_ica_binary)
print('k_NN accuracy for test set: %f' % knn_arousal_ica.score(M_2s_ica, Arousal_Test_ica_binary))
y_true, y_pred = Arousal_Test_ica_binary, knn_arousal_ica.predict(M_2s_ica)
print(classification_report(y_true, y_pred))

knn_valence_ica = knn.fit(X_2s_ica, Valence_Train_ica_binary)
print('k_NN accuracy for test set: %f' % knn_valence_ica.score(M_2s_ica, Valence_Test_ica_binary))
y_true, y_pred = Valence_Test_ica_binary, knn_valence_ica.predict(M_2s_ica)
print(classification_report(y_true, y_pred))

knn_domain_ica = knn.fit(X_2s_ica, Domain_Train_ica_binary)
print('k_NN accuracy for test set: %f' % knn_domain_ica.score(M_2s_ica, Domain_Test_ica_binary))
y_true, y_pred = Domain_Test_ica_binary, knn_domain_ica.predict(M_2s_ica)
print(classification_report(y_true, y_pred))

k_NN accuracy for test set: 0.747551
              precision    recall  f1-score   support

           0       0.87      0.50      0.63     54404
           1       0.71      0.94      0.81     70524

    accuracy                           0.75    124928
   macro avg       0.79      0.72      0.72    124928
weighted avg       0.78      0.75      0.73    124928

k_NN accuracy for test set: 0.700676
              precision    recall  f1-score   support

           0       0.88      0.31      0.46     51136
           1       0.67      0.97      0.79     73792

    accuracy                           0.70    124928
   macro avg       0.77      0.64      0.63    124928
weighted avg       0.75      0.70      0.66    124928

k_NN accuracy for test set: 0.681817
              precision    recall  f1-score   support

           0       0.94      0.17      0.29     47307
           1       0.66      0.99      0.80     77621

    accuracy                           0.68    124928
   macro avg     

In [24]:
knn_domain_ica = knn.fit(X_2s_ica, Domain_Train_ica_binary)
print('k_NN accuracy for test set: %f' % knn_domain_ica.score(M_2s_ica, Domain_Test_ica_binary))
y_true, y_pred = Domain_Test_ica_binary, knn_domain_ica.predict(M_2s_ica)
print(classification_report(y_true, y_pred))

k_NN accuracy for test set: 0.681817
              precision    recall  f1-score   support

           0       0.94      0.17      0.29     47307
           1       0.66      0.99      0.80     77621

    accuracy                           0.68    124928
   macro avg       0.80      0.58      0.54    124928
weighted avg       0.77      0.68      0.60    124928



In [25]:
Val_R_ica = RandomForestRegressor(n_estimators=512, n_jobs=6,random_state=42)
Val_R_ica.fit(X_2s_ica, Valence_Train_ica_binary)
Valence_pred_ica = Val_R_ica.predict(M_2s_ica)
Valence_pred_ica = np.where(Valence_pred_ica >= 0.5, 1, 0)
Ran_valence_ica_accuracy = accuracy_score(Valence_Test_ica_binary, Valence_pred_ica)
Ran_valence_ica_f1 = f1_score(Valence_Test_ica_binary, Valence_pred_ica, average='weighted')
print(Ran_valence_ica_accuracy,Ran_valence_ica_f1)

Aro_R_ica = RandomForestRegressor(n_estimators=512, n_jobs=6,random_state=42)
Aro_R_ica.fit(X_2s_ica, Arousal_Train_ica_binary)
Arousal_pred_ica = Aro_R_ica.predict(M_2s_ica)

Arousal_pred_ica= np.where(Arousal_pred_ica >= 0.5, 1, 0)
Ran_Arousal_ica_accuracy = accuracy_score(Arousal_Test_ica_binary, Arousal_pred_ica)
Ran_Arousal_ica_f1 = f1_score(Arousal_Test_ica_binary, Arousal_pred_ica, average='weighted')

print(Ran_Arousal_ica_accuracy,Ran_Arousal_ica_f1)

Dom_R_ica = RandomForestRegressor(n_estimators=512, n_jobs=6,random_state=42)
Dom_R_ica.fit(X_2s_ica, Domain_Train_ica_binary)
Domain_pred_ica = Dom_R_ica.predict(M_2s_ica)

Domain_pred_ica = np.where(Domain_pred_ica >= 0.5, 1, 0)
Ran_Domain_ica_accuracy = accuracy_score(Domain_Test_ica_binary, Domain_pred_ica)
Ran_Domain_ica_f1 = f1_score(Domain_Test_ica_binary, Domain_pred_ica, average='weighted')

print(Ran_Domain_ica_accuracy,Ran_Domain_ica_f1)

0.9646436347336066 0.964381315593927
0.9772508965163934 0.9771821351286353
0.9542936731557377 0.9536589400684066


In [26]:
with open('C:/Users/wongs/OneDrive/Documents/dissertation/DEAP/out_dwt_data_training_2s.npy', 'rb') as fileTrain:
    X_2s_dwt  = np.load(fileTrain)
    
with open('C:/Users/wongs/OneDrive/Documents/dissertation/DEAP/out_dwt_label_training_2s.npy', 'rb') as fileTrainL:
    Y_2s_dwt  = np.load(fileTrainL)

ss = StandardScaler()
X_2s_dwt = ss.fit_transform(X_2s_dwt)
Z_2s_dwt = np.ravel(Y_2s_dwt[:, [1]])

Arousal_Train_dwt = np.ravel(Y_2s_dwt[:, [0]])
Valence_Train_dwt = np.ravel(Y_2s_dwt[:, [1]])
Domain_Train_dwt = np.ravel(Y_2s_dwt[:, [2]])
Like_Train_dwt = np.ravel(Y_2s_dwt[:, [3]])



with open('C:/Users/wongs/OneDrive/Documents/dissertation/DEAP/out_dwt_data_testing_2s.npy', 'rb') as fileTrain:
    M_2s_dwt  = np.load(fileTrain)
    
with open('C:/Users/wongs/OneDrive/Documents/dissertation/DEAP/out_dwt_label_testing_2s.npy', 'rb') as fileTrainL:
    N_2s_dwt  = np.load(fileTrainL)

M_2s_dwt = ss.transform(M_2s_dwt)
L_2s_dwt = np.ravel(N_2s_dwt[:, [1]])

Arousal_Test_dwt = np.ravel(N_2s_dwt[:, [0]])
Valence_Test_dwt = np.ravel(N_2s_dwt[:, [1]])
Domain_Test_dwt = np.ravel(N_2s_dwt[:, [2]])
Like_Test_dwt = np.ravel(N_2s_dwt[:, [3]])

# Binarize Arousal_Test_2s with threshold 5
Arousal_Test_dwt_binary = np.where(Arousal_Test_dwt >= 5, 1, 0)
Arousal_Train_dwt_binary = np.where(Arousal_Train_dwt >= 5, 1, 0)
# Binarize Valence_Train_2s with threshold 5
Valence_Train_dwt_binary = np.where(Valence_Train_dwt >= 5, 1, 0)
# Binarize Valence_Test_2s with threshold 5
Valence_Test_dwt_binary = np.where(Valence_Test_dwt >= 5, 1, 0)
# Binarize Domain_Test_2s with threshold 5
Domain_Test_dwt_binary = np.where(Domain_Test_dwt >= 5, 1, 0)
# Binarize Like_Train_2s with threshold 5
Domain_Train_dwt_binary = np.where(Domain_Train_dwt >= 5, 1, 0)


In [29]:
svmlinear_arousal_2s_dwt = SVC(kernel='linear',max_iter=100000,random_state=42,verbose=True)
svmlinear_arousal_2s_dwt.fit(X_2s_dwt, Arousal_Train_dwt_binary)
predictions_linear_arousal = svmlinear_arousal_2s_dwt.predict(M_2s_dwt)


svmlinear_arousal_2s_dwt_accuracy = accuracy_score(Arousal_Test_dwt_binary, predictions_linear_arousal)
svmlinear_arousal_2s_dwt_f1 = f1_score(Arousal_Test_dwt_binary, predictions_linear_arousal, average='weighted')
print("Arousal Linear SVM:")
print(f"Accuracy: {svmlinear_arousal_2s_dwt_accuracy * 100:.2f}%")
print(f"F1 Score: {svmlinear_arousal_2s_dwt_f1:.2f}")
svmrbf_arousal_2s_dwt = SVC(kernel='rbf',max_iter=100000,random_state=42,verbose=True)

# Train the models
svmrbf_arousal_2s_dwt.fit(X_2s_dwt, Arousal_Train_dwt_binary)

# Predict on new data
predictions_rbf_arousal = svmrbf_arousal_2s_dwt.predict(M_2s_dwt)


# Evaluate the models
svmrbf_arousal_2s_dwt_accuracy = accuracy_score(Arousal_Test_dwt_binary, predictions_rbf_arousal)
svmrbf_arousal_2s_dwt_f1 = f1_score(Arousal_Test_dwt_binary, predictions_rbf_arousal, average='weighted')



# Print the results
print("Arousal RBF SVM:")
print(f"Accuracy: {svmrbf_arousal_2s_dwt_accuracy * 100:.2f}%")
print(f"F1 Score: {svmrbf_arousal_2s_dwt_f1:.2f}")

[LibSVM]

C:\Users\wongs\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\svm\_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=100000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


Arousal Linear SVM:
Accuracy: 56.16%
F1 Score: 0.42
[LibSVM]Arousal RBF SVM:
Accuracy: 58.47%
F1 Score: 0.50


In [30]:
svmrbf_valence_2s_dwt = SVC(kernel='rbf',max_iter=100000,random_state=42,verbose=True)
svmlinear_valence_2s_dwt = SVC(kernel='linear',max_iter=100000,random_state=42,verbose=True)

svmrbf_valence_2s_dwt.fit(X_2s_dwt, Valence_Train_dwt_binary)
svmlinear_valence_2s_dwt.fit(X_2s_dwt, Valence_Train_dwt_binary)

predictions_rbf_valence = svmrbf_valence_2s_dwt.predict(M_2s_dwt)
predictions_linear_valence = svmlinear_valence_2s_dwt.predict(M_2s_dwt)

svmrbf_valence_2s_dwt_accuracy = accuracy_score(Valence_Test_dwt_binary, predictions_rbf_valence)
svmrbf_valence_2s_dwt_f1 = f1_score(Valence_Test_dwt_binary, predictions_rbf_valence, average='weighted')

svmlinear_valence_2s_dwt_accuracy = accuracy_score(Valence_Test_dwt_binary, predictions_linear_valence)
svmlinear_valence_2s_dwt_f1 = f1_score(Valence_Test_dwt_binary, predictions_linear_valence, average='weighted')

print("Valence RBF SVM:")
print(f"Accuracy: {svmrbf_valence_2s_dwt_accuracy * 100:.2f}%")
print(f"F1 Score: {svmrbf_valence_2s_dwt_f1:.2f}")

print("Valence Linear SVM:")
print(f"Accuracy: {svmlinear_valence_2s_dwt_accuracy * 100:.2f}%")
print(f"F1 Score: {svmlinear_valence_2s_dwt_f1:.2f}")

[LibSVM][LibSVM]

C:\Users\wongs\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\svm\_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=100000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


Valence RBF SVM:
Accuracy: 61.66%
F1 Score: 0.55
Valence Linear SVM:
Accuracy: 59.16%
F1 Score: 0.46


In [31]:
svmrbf_domain_2s_dwt = SVC(kernel='rbf',max_iter=100000,random_state=42,verbose=True)
svmlinear_domain_2s_dwt = SVC(kernel='linear',max_iter=100000,random_state=42,verbose=True)

svmrbf_domain_2s_dwt.fit(X_2s_dwt, Domain_Train_dwt_binary)

svmlinear_domain_2s_dwt.fit(X_2s_dwt, Domain_Train_dwt_binary)

predictions_rbf_domain = svmrbf_domain_2s_dwt.predict(M_2s_dwt)
predictions_linear_domain = svmlinear_domain_2s_dwt.predict(M_2s_dwt)

svmrbf_domain_2s_dwt_accuracy = accuracy_score(Domain_Test_dwt_binary, predictions_rbf_domain)
svmrbf_domain_2s_dwt_f1 = f1_score(Domain_Test_dwt_binary, predictions_rbf_domain, average='weighted')

svmlinear_domain_2s_dwt_accuracy = accuracy_score(Domain_Test_dwt_binary, predictions_linear_domain)
svmlinear_domain_2s_dwt_f1 = f1_score(Domain_Test_dwt_binary, predictions_linear_domain, average='weighted')

print("Domain RBF SVM:")
print(f"Accuracy: {svmrbf_domain_2s_dwt_accuracy * 100:.2f}%")
print(f"F1 Score: {svmrbf_domain_2s_dwt_f1:.2f}")

print("Domain Linear SVM:")
print(f"Accuracy: {svmlinear_domain_2s_dwt_accuracy * 100:.2f}%")
print(f"F1 Score: {svmlinear_domain_2s_dwt_f1:.2f}")

[LibSVM][LibSVM]

C:\Users\wongs\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\svm\_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=100000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


Domain RBF SVM:
Accuracy: 65.13%
F1 Score: 0.58
Domain Linear SVM:
Accuracy: 61.71%
F1 Score: 0.49


In [32]:
n = int(math.sqrt(len(X_2s_dwt)))
knn = neighbors.KNeighborsClassifier(n_neighbors=n, weights='uniform', p=norm)
knn_arousal_dwt = knn.fit(X_2s_dwt, Arousal_Train_dwt_binary)
print('k_NN accuracy for test set: %f' % knn_arousal_dwt.score(M_2s_dwt, Arousal_Test_dwt_binary))
y_true, y_pred = Arousal_Test_dwt_binary, knn_arousal_dwt.predict(M_2s_dwt)
print(classification_report(y_true, y_pred))

knn_valence_dwt = knn.fit(X_2s_dwt, Valence_Train_dwt_binary)
print('k_NN accuracy for test set: %f' % knn_valence_dwt.score(M_2s_dwt, Valence_Test_dwt_binary))
y_true, y_pred = Valence_Test_dwt_binary, knn_valence_dwt.predict(M_2s_dwt)
print(classification_report(y_true, y_pred))

knn_domain_dwt = knn.fit(X_2s_dwt, Domain_Train_dwt_binary)
print('k_NN accuracy for test set: %f' % knn_domain_dwt.score(M_2s_dwt, Domain_Test_dwt_binary))
y_true, y_pred = Domain_Test_dwt_binary, knn_domain_dwt.predict(M_2s_dwt)
print(classification_report(y_true, y_pred))

k_NN accuracy for test set: 0.627016
              precision    recall  f1-score   support

           0       0.60      0.45      0.51     13912
           1       0.64      0.76      0.70     17832

    accuracy                           0.63     31744
   macro avg       0.62      0.61      0.61     31744
weighted avg       0.62      0.63      0.62     31744

k_NN accuracy for test set: 0.661952
              precision    recall  f1-score   support

           0       0.62      0.44      0.51     12950
           1       0.68      0.82      0.74     18794

    accuracy                           0.66     31744
   macro avg       0.65      0.63      0.63     31744
weighted avg       0.66      0.66      0.65     31744

k_NN accuracy for test set: 0.675214
              precision    recall  f1-score   support

           0       0.61      0.39      0.47     12017
           1       0.70      0.85      0.76     19727

    accuracy                           0.68     31744
   macro avg     

In [33]:
Val_R_dwt = RandomForestRegressor(n_estimators=512,random_state=42, n_jobs=6)
Val_R_dwt.fit(X_2s_dwt, Valence_Train_dwt_binary)
Valence_pred_dwt = Val_R_dwt.predict(M_2s_dwt)
Valence_pred_dwt = np.where(Valence_pred_dwt >= 0.5, 1, 0)
Ran_valence_dwt_accuracy = accuracy_score(Valence_Test_dwt_binary, Valence_pred_dwt)
Ran_valence_dwt_f1 = f1_score(Valence_Test_dwt_binary, Valence_pred_dwt, average='weighted')
print(Ran_valence_dwt_accuracy,Ran_valence_dwt_f1)

Aro_R_dwt = RandomForestRegressor(n_estimators=512,random_state=42, n_jobs=6)
Aro_R_dwt.fit(X_2s_dwt, Arousal_Train_dwt_binary)
Arousal_pred_dwt = Aro_R_dwt.predict(M_2s_dwt)

Arousal_pred_dwt= np.where(Arousal_pred_dwt >= 0.5, 1, 0)
Ran_Arousal_dwt_accuracy = accuracy_score(Arousal_Test_dwt_binary, Arousal_pred_dwt)
Ran_Arousal_dwt_f1 = f1_score(Arousal_Test_dwt_binary, Arousal_pred_dwt, average='weighted')

print(Ran_Arousal_dwt_accuracy,Ran_Arousal_dwt_f1)

Dom_R_dwt = RandomForestRegressor(n_estimators=512,random_state=42, n_jobs=6)
Dom_R_dwt.fit(X_2s_dwt, Domain_Train_dwt_binary)
Domain_pred_dwt = Dom_R_dwt.predict(M_2s_dwt)

Domain_pred_dwt = np.where(Domain_pred_dwt >= 0.5, 1, 0)


0.7020854334677419 0.6932902243195519
0.6697328629032258 0.6630123542198529


In [34]:
Ran_Domain_dwt_accuracy = accuracy_score(Domain_Test_dwt_binary, Domain_pred_dwt)
Ran_Domain_dwt_f1 = f1_score(Domain_Test_dwt_binary, Domain_pred_dwt, average='weighted')

print(Ran_Domain_dwt_accuracy,Ran_Domain_dwt_f1)

0.7094254032258065 0.6988190645685566


In [35]:
with open('C:/Users/wongs/OneDrive/Documents/dissertation/DEAP/out_dwt_ica_data_training_2s.npy', 'rb') as fileTrain:
    X_2s_dwt_ica  = np.load(fileTrain)
    
with open('C:/Users/wongs/OneDrive/Documents/dissertation/DEAP/out_dwt_ica_label_training_2s.npy', 'rb') as fileTrainL:
    Y_2s_dwt_ica  = np.load(fileTrainL)

ss = StandardScaler()
X_2s_dwt_ica = ss.fit_transform(X_2s_dwt_ica)
Z_2s_dwt_ica = np.ravel(Y_2s_dwt_ica[:, [1]])

Arousal_Train_dwt_ica = np.ravel(Y_2s_dwt_ica[:, [0]])
Valence_Train_dwt_ica = np.ravel(Y_2s_dwt_ica[:, [1]])
Domain_Train_dwt_ica = np.ravel(Y_2s_dwt_ica[:, [2]])
Like_Train_dwt_ica = np.ravel(Y_2s_dwt_ica[:, [3]])



with open('C:/Users/wongs/OneDrive/Documents/dissertation/DEAP/out_dwt_ica_data_testing_2s.npy', 'rb') as fileTrain:
    M_2s_dwt_ica  = np.load(fileTrain)
    
with open('C:/Users/wongs/OneDrive/Documents/dissertation/DEAP/out_dwt_ica_label_testing_2s.npy', 'rb') as fileTrainL:
    N_2s_dwt_ica  = np.load(fileTrainL)

M_2s_dwt_ica = ss.transform(M_2s_dwt_ica)
L_2s_dwt_ica = np.ravel(N_2s_dwt_ica[:, [1]])

Arousal_Test_dwt_ica = np.ravel(N_2s_dwt_ica[:, [0]])
Valence_Test_dwt_ica = np.ravel(N_2s_dwt_ica[:, [1]])
Domain_Test_dwt_ica = np.ravel(N_2s_dwt_ica[:, [2]])
Like_Test_dwt_ica = np.ravel(N_2s_dwt_ica[:, [3]])

# Binarize Arousal_Test_2s with threshold 5
Arousal_Test_dwt_ica_binary = np.where(Arousal_Test_dwt_ica >= 5, 1, 0)
Arousal_Train_dwt_ica_binary = np.where(Arousal_Train_dwt_ica >= 5, 1, 0)
# Binarize Valence_Train_2s with threshold 5
Valence_Train_dwt_ica_binary = np.where(Valence_Train_dwt_ica >= 5, 1, 0)
# Binarize Valence_Test_2s with threshold 5
Valence_Test_dwt_ica_binary = np.where(Valence_Test_dwt_ica >= 5, 1, 0)
# Binarize Domain_Test_2s with threshold 5
Domain_Test_dwt_ica_binary = np.where(Domain_Test_dwt_ica >= 5, 1, 0)
# Binarize Like_Train_2s with threshold 5
Domain_Train_dwt_ica_binary = np.where(Domain_Train_dwt_ica >= 5, 1, 0)


In [36]:
svmlinear_arousal_2s_dwt_ica = SVC(kernel='linear',max_iter=100000,random_state=42,verbose=True)
svmlinear_arousal_2s_dwt_ica.fit(X_2s_dwt_ica, Arousal_Train_dwt_ica_binary)
predictions_linear_arousal = svmlinear_arousal_2s_dwt_ica.predict(M_2s_dwt_ica)


svmlinear_arousal_2s_dwt_ica_accuracy = accuracy_score(Arousal_Test_dwt_ica_binary, predictions_linear_arousal)
svmlinear_arousal_2s_dwt_ica_f1 = f1_score(Arousal_Test_dwt_ica_binary, predictions_linear_arousal, average='weighted')
print("Arousal Linear SVM:")
print(f"Accuracy: {svmlinear_arousal_2s_dwt_ica_accuracy * 100:.2f}%")
print(f"F1 Score: {svmlinear_arousal_2s_dwt_ica_f1:.2f}")
svmrbf_arousal_2s_dwt_ica = SVC(kernel='rbf',max_iter=100000,random_state=42,verbose=True)

# Train the models
svmrbf_arousal_2s_dwt_ica.fit(X_2s_dwt_ica, Arousal_Train_dwt_ica_binary)

# Predict on new data
predictions_rbf_arousal = svmrbf_arousal_2s_dwt_ica.predict(M_2s_dwt_ica)


# Evaluate the models
svmrbf_arousal_2s_dwt_ica_accuracy = accuracy_score(Arousal_Test_dwt_ica_binary, predictions_rbf_arousal)
svmrbf_arousal_2s_dwt_ica_f1 = f1_score(Arousal_Test_dwt_ica_binary, predictions_rbf_arousal, average='weighted')



# Print the results
print("Arousal RBF SVM:")
print(f"Accuracy: {svmrbf_arousal_2s_dwt_ica_accuracy * 100:.2f}%")
print(f"F1 Score: {svmrbf_arousal_2s_dwt_ica_f1:.2f}")

[LibSVM]

C:\Users\wongs\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\svm\_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=100000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


Arousal Linear SVM:
Accuracy: 56.01%
F1 Score: 0.42
[LibSVM]Arousal RBF SVM:
Accuracy: 60.76%
F1 Score: 0.54


In [37]:
svmrbf_valence_2s_dwt_ica = SVC(kernel='rbf',max_iter=100000,random_state=42,verbose=True)
svmlinear_valence_2s_dwt_ica = SVC(kernel='linear',max_iter=100000,random_state=42,verbose=True)

svmrbf_valence_2s_dwt_ica.fit(X_2s_dwt_ica, Valence_Train_dwt_ica_binary)
svmlinear_valence_2s_dwt_ica.fit(X_2s_dwt_ica, Valence_Train_dwt_ica_binary)

predictions_rbf_valence = svmrbf_valence_2s_dwt_ica.predict(M_2s_dwt_ica)
predictions_linear_valence = svmlinear_valence_2s_dwt_ica.predict(M_2s_dwt_ica)

svmrbf_valence_2s_dwt_ica_accuracy = accuracy_score(Valence_Test_dwt_ica_binary, predictions_rbf_valence)
svmrbf_valence_2s_dwt_ica_f1 = f1_score(Valence_Test_dwt_ica_binary, predictions_rbf_valence, average='weighted')

svmlinear_valence_2s_dwt_ica_accuracy = accuracy_score(Valence_Test_dwt_ica_binary, predictions_linear_valence)
svmlinear_valence_2s_dwt_ica_f1 = f1_score(Valence_Test_dwt_ica_binary, predictions_linear_valence, average='weighted')

print("Valence RBF SVM:")
print(f"Accuracy: {svmrbf_valence_2s_dwt_ica_accuracy * 100:.2f}%")
print(f"F1 Score: {svmrbf_valence_2s_dwt_ica_f1:.2f}")

print("Valence Linear SVM:")
print(f"Accuracy: {svmlinear_valence_2s_dwt_ica_accuracy * 100:.2f}%")
print(f"F1 Score: {svmlinear_valence_2s_dwt_ica_f1:.2f}")

[LibSVM][LibSVM]

C:\Users\wongs\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\svm\_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=100000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


Valence RBF SVM:
Accuracy: 63.01%
F1 Score: 0.55
Valence Linear SVM:
Accuracy: 58.27%
F1 Score: 0.45


In [38]:
svmrbf_domain_2s_dwt_ica = SVC(kernel='rbf',max_iter=100000,random_state=42,verbose=True)
svmlinear_domain_2s_dwt_ica = SVC(kernel='linear',max_iter=100000,random_state=42,verbose=True)

svmrbf_domain_2s_dwt_ica.fit(X_2s_dwt_ica, Domain_Train_dwt_ica_binary)

svmlinear_domain_2s_dwt_ica.fit(X_2s_dwt_ica, Domain_Train_dwt_ica_binary)

predictions_rbf_domain = svmrbf_domain_2s_dwt_ica.predict(M_2s_dwt_ica)
predictions_linear_domain = svmlinear_domain_2s_dwt_ica.predict(M_2s_dwt_ica)

svmrbf_domain_2s_dwt_ica_accuracy = accuracy_score(Domain_Test_dwt_ica_binary, predictions_rbf_domain)
svmrbf_domain_2s_dwt_ica_f1 = f1_score(Domain_Test_dwt_ica_binary, predictions_rbf_domain, average='weighted')

svmlinear_domain_2s_dwt_ica_accuracy = accuracy_score(Domain_Test_dwt_ica_binary, predictions_linear_domain)
svmlinear_domain_2s_dwt_ica_f1 = f1_score(Domain_Test_dwt_ica_binary, predictions_linear_domain, average='weighted')

print("Domain RBF SVM:")
print(f"Accuracy: {svmrbf_domain_2s_dwt_ica_accuracy * 100:.2f}%")
print(f"F1 Score: {svmrbf_domain_2s_dwt_ica_f1:.2f}")

print("Domain Linear SVM:")
print(f"Accuracy: {svmlinear_domain_2s_dwt_ica_accuracy * 100:.2f}%")
print(f"F1 Score: {svmlinear_domain_2s_dwt_ica_f1:.2f}")

[LibSVM][LibSVM]

C:\Users\wongs\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\svm\_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=100000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


Domain RBF SVM:
Accuracy: 63.28%
F1 Score: 0.51
Domain Linear SVM:
Accuracy: 61.42%
F1 Score: 0.48


In [39]:
n = int(math.sqrt(len(X_2s_dwt_ica)))
knn = neighbors.KNeighborsClassifier(n_neighbors=n, weights='uniform', p=norm)
knn_arousal_dwt_ica = knn.fit(X_2s_dwt_ica, Arousal_Train_dwt_ica_binary)
print('k_NN accuracy for test set: %f' % knn_arousal_dwt_ica.score(M_2s_dwt_ica, Arousal_Test_dwt_ica_binary))
y_true, y_pred = Arousal_Test_dwt_ica_binary, knn_arousal_dwt_ica.predict(M_2s_dwt_ica)
print(classification_report(y_true, y_pred))

knn_valence_dwt_ica = knn.fit(X_2s_dwt_ica, Valence_Train_dwt_ica_binary)
print('k_NN accuracy for test set: %f' % knn_valence_dwt_ica.score(M_2s_dwt_ica, Valence_Test_dwt_ica_binary))
y_true, y_pred = Valence_Test_dwt_ica_binary, knn_valence_dwt_ica.predict(M_2s_dwt_ica)
print(classification_report(y_true, y_pred))

knn_domain_dwt_ica = knn.fit(X_2s_dwt_ica, Domain_Train_dwt_ica_binary)
print('k_NN accuracy for test set: %f' % knn_domain_dwt_ica.score(M_2s_dwt_ica, Domain_Test_dwt_ica_binary))
y_true, y_pred = Domain_Test_dwt_ica_binary, knn_domain_dwt_ica.predict(M_2s_dwt_ica)
print(classification_report(y_true, y_pred))

k_NN accuracy for test set: 0.582800
              precision    recall  f1-score   support

           0       0.64      0.10      0.18      6829
           1       0.58      0.96      0.72      8787

    accuracy                           0.58     15616
   macro avg       0.61      0.53      0.45     15616
weighted avg       0.61      0.58      0.48     15616

k_NN accuracy for test set: 0.598809
              precision    recall  f1-score   support

           0       0.74      0.04      0.08      6433
           1       0.60      0.99      0.74      9183

    accuracy                           0.60     15616
   macro avg       0.67      0.52      0.41     15616
weighted avg       0.66      0.60      0.47     15616

k_NN accuracy for test set: 0.619621
              precision    recall  f1-score   support

           0       0.75      0.01      0.01      5967
           1       0.62      1.00      0.76      9649

    accuracy                           0.62     15616
   macro avg     

In [40]:
Val_R_dwt_ica = RandomForestRegressor(n_estimators=512, n_jobs=6,random_state=42)
Val_R_dwt_ica.fit(X_2s_dwt_ica, Valence_Train_dwt_ica_binary)
Valence_pred_dwt_ica = Val_R_dwt.predict(M_2s_dwt_ica)
Valence_pred_dwt_ica = np.where(Valence_pred_dwt_ica >= 0.5, 1, 0)
Ran_valence_dwt_ica_accuracy = accuracy_score(Valence_Test_dwt_ica_binary, Valence_pred_dwt_ica)
Ran_valence_dwt_ica_f1 = f1_score(Valence_Test_dwt_ica_binary, Valence_pred_dwt_ica, average='weighted')
print(Ran_valence_dwt_ica_accuracy,Ran_valence_dwt_ica_f1)

Aro_R_dwt_ica = RandomForestRegressor(n_estimators=512, n_jobs=6,random_state=42)
Aro_R_dwt_ica.fit(X_2s_dwt_ica, Arousal_Train_dwt_ica_binary)
Arousal_pred_dwt_ica = Aro_R_dwt_ica.predict(M_2s_dwt_ica)

Arousal_pred_dwt_ica= np.where(Arousal_pred_dwt_ica >= 0.5, 1, 0)
Ran_Arousal_dwt_ica_accuracy = accuracy_score(Arousal_Test_dwt_ica_binary, Arousal_pred_dwt_ica)
Ran_Arousal_dwt_ica_f1 = f1_score(Arousal_Test_dwt_ica_binary, Arousal_pred_dwt_ica, average='weighted')

print(Ran_Arousal_dwt_ica_accuracy,Ran_Arousal_dwt_ica_f1)

Dom_R_dwt_ica = RandomForestRegressor(n_estimators=512, n_jobs=6,random_state=42)
Dom_R_dwt_ica.fit(X_2s_dwt_ica, Domain_Train_dwt_ica_binary)
Domain_pred_dwt_ica = Dom_R_dwt_ica.predict(M_2s_dwt_ica)

Domain_pred_dwt_ica = np.where(Domain_pred_dwt_ica >= 0.5, 1, 0)
Ran_Domain_dwt_ica_accuracy = accuracy_score(Domain_Test_dwt_ica_binary, Domain_pred_dwt_ica)
Ran_Domain_dwt_ica_f1 = f1_score(Domain_Test_dwt_ica_binary, Domain_pred_dwt_ica, average='weighted')

print(Ran_Domain_dwt_ica_accuracy,Ran_Domain_dwt_ica_f1)

0.49103483606557374 0.491316600827417
0.6070056352459017 0.5230136149842203
0.6382556352459017 0.5251885179439337
